# Installs, Imports and Creds

In [1]:
%pip install spotipy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import spotipy
import pandas as pd
import config
from pprint import pprint


In [ ]:
# display all rows in dataframe
pd.set_option('display.max_rows', None)

# grab current date
current_date = pd.Timestamp.today().date()


In [2]:
# Authenticate with Spotify
user_auth = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=config.client,
    client_secret=config.secret,
    redirect_uri=config.redirect,
    scope=config.scope
))

# Get current user's information
current_user = user_auth.current_user()


# Current User Top Aritsts

In [ ]:
# time_range:  Valid options are: 
# 'short_term': Last 4 weeks
# 'medium_term': Last 6 months
# 'long_term': Overall listening history
term = 'medium_term'

# Get current user's recently played tracks
current_user_top_artists = user_auth.current_user_top_artists(time_range = term, limit=50)


# empty lists for all the important fields 
user_artist__name = []
user_artist__genres = []
user_artist__popularity = []
user_artist__followers = []

# return the top 50 artists
for artist in current_user_top_artists['items'][:50]:

    # grab artist name and append to list
    artist_name = artist['name']
    user_artist__name.append(artist_name)

    # grab artist genre and append to list
    artist_genres = artist['genres']
    user_artist__genres.append(artist_genres)
 
     # grab artist popularity and append to list
    artist_popularity = artist['popularity']
    user_artist__popularity.append(artist_popularity)

     # grab artist followers and append to list
    artist_followers = artist['followers']['total']
    user_artist__followers.append(artist_followers)


# dataframe used for visualization
user_top_artist_df = pd.DataFrame(columns = ['run_date', 'time_range', 'name', 'genres', 'popularity', 'followers'])


# appending data to dataframe for visualization, prints out all the songs in the playlist
user_top_artist_df['name'] = user_artist__name
user_top_artist_df['followers'] = user_artist__followers
user_top_artist_df['genres'] = user_artist__genres
user_top_artist_df['popularity'] = user_artist__popularity
user_top_artist_df['run_date'] = current_date
user_top_artist_df['time_range'] = term

user_top_artist_df

In [ ]:
# saving data to csv
user_top_filename = f'csvs/top_artists__{term}__{current_date}.csv'
user_top_artist_df.to_csv(user_top_filename, index=False)

csvs/top_artists_6months__2024-11-17.csv


# Current User Saved Albums

In [ ]:
# empty lists for appending album data retrieved from api to dataframe
user_album = []

# grabbing first batch user saved albums from spotify api
current_user_saved_albums = user_auth.current_user_saved_albums()
user_album.extend(current_user_saved_albums['items'])

# grabbing rest of user saved albums data with pagination
while current_user_saved_albums['next']:
    current_user_saved_albums = user_auth.next(current_user_saved_albums)
    user_album.extend(current_user_saved_albums['items'])



# empty lists for all the important fields 
user_album__artist = []
user_album__name = []
user_album__popularity = []
user_album__releaseDate = []
user_album__savedDatetime = []
user_album__url = []

# iterate through and append key datapoints
for item in user_album:

    # grab album artist and append to list
    album__artist = item['album']['artists']
    user_album__artist.append(album__artist[0]['name'])

    # grab album name and append to list
    album__name = item['album']['name']
    user_album__name.append(album__name)

    # grab album popularity and append to list
    album__popularity = item['album']['popularity']
    user_album__popularity.append(album__popularity)

    # grab album release date and append to list
    album__releaseDate = item['album']['release_date']
    user_album__releaseDate.append(album__releaseDate)

    # grab album saved date and append to list
    album__savedDatetime = item['added_at']
    user_album__savedDatetime.append(album__savedDatetime)

    # grab album link and append to list
    album__url = item['album']['external_urls']['spotify']
    user_album__url.append(album__url)


# dataframe used for visualization
user_album_df = pd.DataFrame(columns = ['album', 'artist', 'releaseDate', 'savedDate', 'popularity', 'link'])

# appending data to df for visualization, prints out all the songs in the playlist
user_album_df['album'] = user_album__name
user_album_df['artist'] = user_album__artist
user_album_df['link'] = user_album__url
user_album_df['popularity'] = user_album__popularity
user_album_df['releaseDate'] = user_album__releaseDate
user_album_df['savedDatetime'] = user_album__savedDatetime

# converting saved datetime to date
user_album_df['savedDatetime'] = pd.to_datetime(user_album_df['savedDatetime'])
user_album_df['savedDate'] =  user_album_df['savedDatetime'].dt.date
user_album_df

# sorting df by artist
user_album_df_sorted = user_album_df.sort_values(by='artist', ascending=True)
user_album_df_sorted


In [ ]:
# saving data to csv
user_album_filename = f'csvs/saved_albums__{current_date}.csv'
user_album_df_sorted.to_csv(user_album_filename, index=False)